# Steps Analysis (02/17/17 - 04/10/19)

In [20]:
from app import FXYRS
from stats_functions_class import MainStats
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd
import numpy as np
init_notebook_mode(connected=True)

In [7]:
fxyrs = FXYRS('../data/export.xml')
floors_df, dist_df, steps_df, combined_df = fxyrs.clean()

In [21]:
combined_df.head()

,start_date,end_date,num_steps,tot_dist,num_floors,ft_per_step,avg_steps,avg_dist,avg_flrs
0,2017-02-17,2017-02-17,812.0,0.342537,1.0,2.227332,812.00,0.342537,1.000000
1,2017-02-18,2017-02-18,2668.0,1.048047,1.0,2.074097,1740.00,0.695292,1.000000
2,2017-02-19,2017-02-19,2991.0,1.249733,3.0,2.206148,2157.00,0.880106,1.666667
3,2017-02-20,2017-02-20,2812.0,1.178545,0.0,2.212915,2320.75,0.954715,1.250000
4,2017-02-21,2017-02-21,2819.0,1.110353,1.0,2.079696,2420.40,0.985843,1.200000


In [22]:
Mw = MainStats(combined_df, 'week')
Mm = MainStats(combined_df, 'month')
Mdw = MainStats(combined_df, 'day of week')
Mdm = MainStats(combined_df, 'day of month')

week_df = Mw.run_all()
month_df = Mm.run_all()
dow_df = Mdw.run_all()
dom_df = Mdm.run_all()

In [23]:
week_df

,week_range,std_steps,std_dist,std_flrs,std_fps,mean_steps,mean_dist,mean_flrs,mean_fps,avg_steps,avg_dist,avg_flrs,steps_score,dist_score,floors_score
0,02.12.17,1312.4,0.498872,0.000000,0.108353,1740.0,0.70,1.00,2.150714,812.0,0.342537,1.000000,2.7,2.8,0.0
1,02.19.17,1065.4,0.416894,1.214986,0.077284,3439.4,1.42,1.86,2.185161,1740.0,0.695292,1.000000,12.8,14.0,3.4
2,02.26.17,791.7,0.293838,1.112697,0.061863,2623.2,1.04,1.71,2.113874,2157.0,0.880106,1.666667,10.0,10.7,3.2
3,03.05.17,1734.3,0.705862,0.577350,0.117129,3099.6,1.25,1.00,2.121595,2320.8,0.954715,1.250000,6.4,6.4,2.1
4,03.12.17,1119.9,0.482824,1.902379,0.068348,3650.3,1.44,2.57,2.075791,2420.4,0.985843,1.200000,13.8,12.5,4.2
5,03.19.17,1150.8,0.477229,1.112697,0.079690,3226.3,1.29,2.29,2.090588,2443.5,1.009125,1.166667,10.5,10.0,5.6
6,03.26.17,1313.1,0.534630,2.000000,0.053690,3581.7,1.43,3.00,2.105391,2722.4,1.118263,1.428571,11.3,11.1,5.4
7,04.02.17,1416.1,0.575413,2.870208,0.033059,3515.4,1.39,4.29,2.077172,2764.6,1.137721,1.500000,10.1,9.7,7.7
8,04.09.17,1481.7,0.577476,1.069045,0.031443,4225.1,1.66,3.14,2.078746,3061.7,1.257063,1.666667,13.9,13.9,11.1
9,04.16.17,2126.4,0.869204,2.563480,0.065534,4580.0,1.83,3.29,2.095672,2914.8,1.195752,1.600000,11.4,11.1,5.1


In [27]:
def pltly_dict(ttl, lgnd_ornt, yxs_ttl, y2xs_ttl, yxs_type='linear'):
    fdict = {
        'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
        'font': {'color': 'white'}, 'title': ttl,
        'legend': {'orientation': 'h', 'x': lgnd_ornt[0], 'y': lgnd_ornt[1]},
                  'yaxis': {'title': yxs_ttl, 'type': yxs_type},
                  'yaxis2': {'title': y2xs_ttl, 'titlefont': {'color': 'orange'},
                             'tickfont': {'color': 'orange'}, 'overlaying': 'y', 'side': 'right'}}
    return fdict

In [35]:
b_ws = go.Scatter(x=combined_df.start_date, y=combined_df.num_steps,
              name='Daily Average Steps', yaxis='y1', marker={'color': 'steelblue'})

b_wsa = go.Scatter(x=combined_df.start_date, y=combined_df.avg_steps,
                   name='Daily Average Steps', yaxis='y1', marker={'color': 'white'})

# sl_ws = go.Scatter(x=combined_df.week_range, y=combined_df.steps_score,
#                    name='Steps Score', yaxis='y2', marker={'color': 'orange'})

wsd = pltly_dict('Daily Steps and Score', [.25, 1.1], 'Daily Average Steps', 'Steps Score')

l_ws = go.Layout(wsd)

d_ws = [b_wsa, b_ws, sl_ws]
f_ws = go.Figure(data=d_ws, layout=l_ws)
iplot(f_ws)

In [33]:
b_ws = go.Bar(x=week_df.week_range, y=week_df.mean_steps,
              name='Daily Average Steps', yaxis='y1', marker={'color': 'steelblue'})

b_wsa = go.Scatter(x=week_df.week_range, y=week_df.avg_steps,
                   name='Daily Average Steps', yaxis='y1', marker={'color': 'white'})

sl_ws = go.Scatter(x=week_df.week_range, y=week_df.steps_score,
                   name='Steps Score', yaxis='y2', marker={'color': 'orange'})

wsd = pltly_dict('Weekly Steps and Score', [.25, 1.1], 'Daily Average Steps', 'Steps Score')

l_ws = go.Layout(wsd)

d_ws = [b_wsa, b_ws, sl_ws]
f_ws = go.Figure(data=d_ws, layout=l_ws)
iplot(f_ws)

In [25]:
b_wd = go.Bar(x=week_df.week_range, y=week_df.mean_dist,
              name='Avg Distance Walked (Miles)', yaxis='y1')

sl_wd = go.Scatter(x=week_df.week_range, y=week_df.dist_score,
                        name='Distance Walked Score', yaxis='y2')

wdd = pltly_dict('Weekly Distance Walked (Miles) and Score', [.25, 1.1],
                 'Avg Distance Walked', 'Distance Walked Score')

l_wd = go.Layout(wdd)

dwd = [b_wd, sl_wd]
fig = go.Figure(data=dwd, layout=l_wd)
iplot(fig)

In [26]:
b_wc = go.Bar(x=week_df.week_range, y=week_df.mean_flrs,
              name='Avg Floors Climbed', yaxis='y1')

sl_wc = go.Scatter(x=week_df.week_range, y=week_df.floors_score,
                        name='Floors Climbed Score', yaxis='y2')
wcd = pltly_dict('Weekly Floors Climbed and Score', [.29, 1.1],
                 'Avg Floors Climbed', 'Floors Climbed Score')

l_wc = go.Layout(wcd)

dwc = [b_wc, sl_wc]
fig = go.Figure(data=dwc, layout=l_wc)
iplot(fig)

In [11]:
b_ms = go.Bar(x=month_df.month_year, y=month_df.mean_steps,
              name='Avg Steps', yaxis='y1')

sl_ms = go.Scatter(x=month_df.month_year, y=month_df.steps_score,
                        name='Steps Score', yaxis='y2')
msd = pltly_dict('Monthly Steps and Score', [.32, 1.1], 'Daily Average Steps', 'Steps Score')

l_ws = go.Layout(wsd)

d_ws = [b_ws, sl_ws]
f_ws = go.Figure(data=d_ws, layout=l_ws)
iplot(f_ws)
layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Monthly Steps and Score', 'yaxis': {'title': 'Avg Steps'},
                    'legend': {'orientation': 'h', 'x': .36, 'y': 1.1},
                    'yaxis2': {'title': 'Steps Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y', 'side': 'right'},
                    'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [21]:
bars = go.Bar(x=month_df.month_year, y=month_df.mean_steps,
              name='Avg Steps', yaxis='y1', marker={'color': 'steelblue'})

score_line = go.Scatter(x=month_df.month_year, y=month_df.steps_score,
                        name='Steps Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Monthly Steps and Score', 'yaxis': {'title': 'Avg Steps'},
                    'legend': {'orientation': 'h', 'x': .36, 'y': 1.1},
                    'yaxis2': {'title': 'Steps Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y', 'side': 'right'},
                    'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [13]:
bars = go.Bar(x=month_df.month_year, y=month_df.mean_dist,
              name='Avg Distance Walked (Miles)', yaxis='y1')

score_line = go.Scatter(x=month_df.month_year, y=month_df.dist_score,
                        name='Distance Walked Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Monthly Distance Walked (Miles) and Score',
                    'legend': {'orientation': 'h', 'x': .25, 'y': 1.1},
                    'yaxis': {'title': 'Avg Distance Walked (Miles)'},
                    'yaxis2': {'title': 'Distance Walked Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y', 'side': 'right'},
                    'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [14]:
bars = go.Bar(x=month_df.month_year, y=month_df.mean_flrs,
              name='Avg Floors Climbed', yaxis='y1')

score_line = go.Scatter(x=month_df.month_year, y=month_df.floors_score,
                        name='Floors Climbed Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Monthly Floors Climbed and Score',
                    'legend': {'orientation': 'h', 'x': .29, 'y': 1.1},
                    'yaxis': {'title': 'Avg Floors Climbed'},
                    'yaxis2': {'title': 'Floors Climbed Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y', 'side': 'right'},
                    'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [15]:
bars = go.Bar(x=dow_df.weekday_year, y=dow_df.mean_steps,
              name='Avg Steps', yaxis='y1')

score_line = go.Scatter(x=dow_df.weekday_year, y=dow_df.steps_score,
                        name='Steps Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Steps and Score by the Day of the Week ', 'yaxis': {'title': 'Avg Steps'},
                    'legend': {'orientation': 'h', 'x': .35, 'y': 1.1},
                    'yaxis2': {'title': 'Steps Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y',
                               'side': 'right'},
                    'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [16]:
bars = go.Bar(x=dow_df.weekday_year, y=dow_df.mean_dist,
              name='Avg Distance Walked (Miles)', yaxis='y1')

score_line = go.Scatter(x=dow_df.weekday_year, y=dow_df.dist_score,
                        name='Distance Walked Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'legend': {'orientation': 'h', 'x': .26, 'y': 1.1},
                    'title': 'Distance Walked (Miles) and Score by the Day of the Week ',
                    'yaxis': {'title': 'Avg Distance Walked (Miles)'},
                    'yaxis2': {'title': 'Distance Walked Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                    'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [17]:
bars = go.Bar(x=dow_df.weekday_year, y=dow_df.mean_flrs,
              name='Avg Floors Climbed', yaxis='y1')

score_line = go.Scatter(x=dow_df.weekday_year, y=dow_df.floors_score,
                        name='Floors Climbed Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'legend': {'orientation': 'h', 'x': .29, 'y': 1.1},
                    'title': 'Floors Climbed and Score by the Day of the Week ',
                    'yaxis': {'title': 'Avg Floors Climbed'},
                    'yaxis2': {'title': 'Floors Climbed Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                    'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [18]:
bars = go.Bar(x=dom_df['date_year'], y=dom_df.mean_steps,
              name='Avg Steps', yaxis='y1')

score_line = go.Scatter(x=dom_df['date_year'], y=dom_df.steps_score,
                        name='Steps Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Steps and Score by the Day of the Month ', 'yaxis': {'title': 'Avg Steps'},
                    'legend': {'orientation': 'h', 'x': .35, 'y': 1.1},
                    'yaxis2': {'title': 'Steps Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y',
                               'side': 'right', 'type': 'log'},
                    'xaxis': {'type': 'category'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [19]:
bars = go.Bar(x=dom_df.date_year, y=dom_df.mean_dist,
              name='Avg Distance Walked (Miles)', yaxis='y1')

score_line = go.Scatter(x=dom_df.date_year, y=dom_df.dist_score,
                        name='Distance Walked Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'legend': {'orientation': 'h', 'x': .25, 'y': 1.1},
                    'title': 'Distance Walked (Miles) and Score by the Day of the Month ',
                    'yaxis': {'title': 'Avg Distance Walked (Miles)'},
                    'yaxis2': {'title': 'Distance Walked Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y', 'type': 'log',
                               'side': 'right'},
                    'xaxis': {'type': 'category'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [20]:
bars = go.Bar(x=dom_df.date_year, y=dom_df.mean_flrs,
              name='Avg Floors Climbed', yaxis='y1')

score_line = go.Scatter(x=dom_df.date_year, y=dom_df.floors_score,
                        name='Floors Climbed Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'legend': {'orientation': 'h', 'x': .29, 'y': 1.1},
                    'title': 'Floors Climbed and Score by the Day of the Month ',
                    'yaxis': {'title': 'Avg Floors Climbed'},
                    'yaxis2': {'title': 'Floors Climbed Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                    'xaxis': {'type': 'category'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)